<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="assets/LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong>CC3106 - Responsible AI
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Proyecto 1:</strong> Identificación y Mitigación de Sesgos en Modelos de Machine Learning
</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autor:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Mario Antonio Guerra Morales - <strong>21008</strong></li>
        <li>Kristopher Javier Alvarado López - <strong>21188</strong></li>
        <li>Daniel Adolfo Sarmiento Peralta - <strong>231105</strong></li>
      </ul>
    </td>
  </tr>
</table>

In [27]:
# --- Imports y configuración global (PRIMERA CELDA) ---
import os
from pathlib import Path

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import pandas as pd
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

FIG_DIR = Path("docs/assets/figures")
DL_DIR = Path("docs/assets/downloads")
FIG_DIR.mkdir(parents=True, exist_ok=True)
DL_DIR.mkdir(parents=True, exist_ok=True)

# (Opcional) estilos y tamaño por defecto
plt.rcParams["figure.figsize"] = (6, 4)
plt.rcParams["figure.dpi"] = 120

In [28]:
# !pip install ucimlrepo

## 🔍 Análisis Exploratorio 

In [29]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

In [30]:
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
print(adult.variables) 


              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

In [31]:
print(X)
print(y)

       age         workclass  fnlwgt  education  education-num  \
0       39         State-gov   77516  Bachelors             13   
1       50  Self-emp-not-inc   83311  Bachelors             13   
2       38           Private  215646    HS-grad              9   
3       53           Private  234721       11th              7   
4       28           Private  338409  Bachelors             13   
...    ...               ...     ...        ...            ...   
48837   39           Private  215419  Bachelors             13   
48838   64               NaN  321403    HS-grad              9   
48839   38           Private  374983  Bachelors             13   
48840   44           Private   83891  Bachelors             13   
48841   35      Self-emp-inc  182148  Bachelors             13   

           marital-status         occupation    relationship  \
0           Never-married       Adm-clerical   Not-in-family   
1      Married-civ-spouse    Exec-managerial         Husband   
2              

In [32]:
df = pd.concat([X, y], axis=1)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K.
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


## Análisis Exploratorio y limpieza de datos

In [33]:
df.info()
df.describe(include="all")
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64

In [34]:
import matplotlib.pyplot
# matplotlib.use("Agg")  # backend "headless" para CI
import matplotlib.pyplot as plt


df['income'].value_counts(normalize=True).plot(kind="bar")
matplotlib.pyplot.title("Distribución de Income (>50K vs <=50K)")
plt.show()

# --- LIMPIEZA DE LA VARIABLE income ---

# Revisar valores únicos originales
print("Valores únicos originales en income:")
print(df['income'].unique())

# 1. Convertir todo a mayúsculas para evitar problemas de 'K' vs 'k'
# 2. Eliminar espacios en blanco extra al inicio/fin (strip)
# 3. Eliminar puntos '.' que aparecen en algunos valores
df['income'] = df['income'].str.upper().str.strip().str.replace('.', '', regex=False)

# Revisar valores únicos después de la limpieza
print("\nValores únicos después de limpieza:")
print(df['income'].unique())

# --- VISUALIZACIÓN DE LA DISTRIBUCIÓN ---
df['income'].value_counts(normalize=True).plot(kind="bar")
plt.figure(figsize=(5, 3))
plt.title("Distribución de Income (>50K vs <=50K)")
plt.ylabel("Proporción")
plt.xlabel("Income")
plt.show()


Valores únicos originales en income:
['<=50K' '>50K' '<=50K.' '>50K.']

Valores únicos después de limpieza:
['<=50K' '>50K']


C:\Users\daher\AppData\Local\Temp\ipykernel_36700\3581098671.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\daher\AppData\Local\Temp\ipykernel_36700\3581098671.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [35]:
# =========================================================
# LIMPIEZA DE VARIABLES CATEGÓRICAS (Adult Census Income)
# =========================================================

import numpy as np
import pandas as pd

# -----------------------------
# 1) Normalizar la variable TARGET (income)
#    - Mayúsculas, sin espacios extra, sin puntos
#    - Validar que queden solo dos categorías
# -----------------------------
if 'income' in df.columns:
    print("Valores únicos originales en income:", df['income'].unique())
    df['income'] = (
        df['income']
        .astype(str)
        .str.upper()
        .str.strip()
        .str.replace('.', '', regex=False)   # elimina puntos al final
    )
    # A veces aparece '>50k' en minúscula; ya quedó en mayúsculas
    # Verificación
    print("Valores únicos tras limpieza income:", df['income'].unique())
    assert set(df['income'].unique()) <= {">50K", "<=50K"}, "Income tiene valores fuera de {>50K, <=50K}"

# -----------------------------
# 2) Lista de variables categóricas a limpiar
#    (además de sex, race, education que ya exploraste)
# -----------------------------
cat_cols = ["workclass", "marital-status", "occupation", "relationship", "native-country"]

# -----------------------------
# 3) Reemplazar "?" por NaN y limpiar espacios
# -----------------------------
for col in cat_cols:
    # Mostrar un vistazo previo
    print(f"\n[{col}] valores únicos (antes):", df[col].unique()[:15], "...")
    # Limpieza genérica
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()                # quita espacios al inicio/fin
        .replace({"?": np.nan})     # "?" -> NaN (faltante real)
    )
    # Mostrar un vistazo posterior
    print(f"[{col}] valores únicos (después):", df[col].dropna().unique()[:15], "...")
    print(f"[{col}] faltantes después:", df[col].isna().sum())

# -----------------------------
# 4) Reagrupar países poco frecuentes en "Other"
#    - Define el umbral por conteo ABSOLUTO o por PROPORCIÓN
#    - Recomendación: usar conteo para Adult (dataset grande)
# -----------------------------
COUNTRY_MIN_COUNT = 200  # <-- puedes ajustar: 100/200/500 según el nivel de agregación deseado

if "native-country" in df.columns:
    country_counts = df["native-country"].value_counts(dropna=True)
    common_countries = country_counts[country_counts >= COUNTRY_MIN_COUNT].index

    # Mantén NaN como NaN; las categorías raras se etiquetan como "Other"
    df["native-country"] = df["native-country"].where(df["native-country"].isna() | df["native-country"].isin(common_countries), "Other")

    print("\nResumen native-country tras agrupación:")
    print(df["native-country"].value_counts(dropna=False).head(15))
    print(f"Total países 'comunes': {len(common_countries)}")
    print(f"Registros etiquetados como 'Other': {(df['native-country'] == 'Other').sum()}")

# -----------------------------
# 5) Chequeo rápido de consistencia
#    - Top categorías por columna
# -----------------------------
print("\n=== Top categorías por variable (post-limpieza) ===")
for col in cat_cols:
    vc = df[col].value_counts(dropna=False).head(10)
    print(f"\n{col}:\n{vc}")

# -----------------------------
# 6) (Opcional) Guardar dataset limpio para seguir EDA/modelado
#    Usa el DL_DIR que definiste al inicio del notebook
# -----------------------------
try:
    out_path = DL_DIR / "adult_clean.csv"
    df.to_csv(out_path, index=False)
    print(f"\n✅ Dataset limpio guardado en: {out_path}")
except Exception as e:
    print("\nNota: no se guardó el CSV porque DL_DIR no existe en este entorno. Error:", e)


Valores únicos originales en income: ['<=50K' '>50K']
Valores únicos tras limpieza income: ['<=50K' '>50K']

[workclass] valores únicos (antes): ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked' nan] ...
[workclass] valores únicos (después): ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'Self-emp-inc' 'Without-pay' 'Never-worked' 'nan'] ...
[workclass] faltantes después: 1836

[marital-status] valores únicos (antes): ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed'] ...
[marital-status] valores únicos (después): ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed'] ...
[marital-status] faltantes después: 0

[occupation] valores únicos (antes): ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving

In [36]:
# --- GRÁFICAS EN GRID 2x2 ---
fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # 2 filas x 2 columnas

# --- Sexo ---
df['sex'].value_counts(normalize=True).plot(kind="bar", ax=axes[0,0])
axes[0,0].set_title("Distribución de Sex")
axes[0,0].set_ylabel("Proporción")

# --- Raza ---
df['race'].value_counts(normalize=True).plot(kind="bar", ax=axes[0,1])
axes[0,1].set_title("Distribución de Race")
axes[0,1].set_ylabel("Proporción")

# --- Educación ---
df['education'].value_counts().plot(kind="barh", ax=axes[1,0])
axes[1,0].set_title("Distribución de Education")
axes[1,0].set_xlabel("Frecuencia")

# --- Horas por semana ---
df['hours-per-week'].hist(bins=30, ax=axes[1,1])
axes[1,1].set_title("Distribución de horas trabajadas por semana")
axes[1,1].set_xlabel("Horas")
axes[1,1].set_ylabel("Frecuencia")

plt.tight_layout()  # Ajusta los espacios entre gráficos
plt.show()


C:\Users\daher\AppData\Local\Temp\ipykernel_36700\1391777974.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [37]:
# --- LISTA DE VARIABLES CATEGÓRICAS A REVISAR ---
categorical_vars = ["workclass", "marital-status", "occupation", "relationship", "native-country"]

for col in categorical_vars:
    print(f"\n--- {col.upper()} ---")
    print("Valores únicos:")
    print(df[col].unique())  # revisar categorías diferentes
    
    print("\nFrecuencia (top 10):")
    print(df[col].value_counts(dropna=False).head(10))  # top categorías más frecuentes
    
    print("-"*50)

fig, axes = plt.subplots(3, 2, figsize=(14, 12))
axes = axes.flatten()

for i, col in enumerate(categorical_vars):
    df[col].value_counts(normalize=True).head(10).plot(kind="bar", ax=axes[i])
    axes[i].set_title(f"Distribución de {col} (Top 10)")
    axes[i].set_ylabel("Proporción")

# Ocultar subplot vacío (porque tenemos 5 variables, no 6)
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()



--- WORKCLASS ---
Valores únicos:
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' nan
 'Self-emp-inc' 'Without-pay' 'Never-worked' 'nan']

Frecuencia (top 10):
workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
NaN                  1836
Self-emp-inc         1695
Federal-gov          1432
nan                   963
Without-pay            21
Never-worked           10
Name: count, dtype: int64
--------------------------------------------------

--- MARITAL-STATUS ---
Valores únicos:
['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']

Frecuencia (top 10):
marital-status
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: count, dtype: int64
-----------------------------------

C:\Users\daher\AppData\Local\Temp\ipykernel_36700\331387201.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# Identificación de posibles sesgos

🔎 1. Sexo (Sex)

Distribución: ~67% hombres, 33% mujeres.

Posible sesgo:

- El dataset está desbalanceado → el modelo puede aprender más de patrones masculinos que femeninos.

- Estudios previos muestran que la tasa de ingresos >50K es mucho mayor en hombres.

- Riesgo: que el modelo aprenda una relación “ser hombre → más ingresos”.

🔎 2. Raza (Race)

Distribución: ~85% blancos, minorías con muy poca representación (ej. Amer-Indian-Eskimo <1%).

Posible sesgo:

- Los grupos minoritarios están subrepresentados → el modelo tendrá poca información para ellos.

- Esto puede generar predicciones menos confiables y potencialmente discriminatorias hacia esos grupos.

🔎 3. Educación (Education)

Distribución: fuerte concentración en HS-grad, Some-college y Bachelors.

Posible sesgo:

- Los niveles bajos de educación tienen muy pocos ejemplos → el modelo puede sobreajustar.

- Esto refuerza la idea de que “más educación → más ingresos”, que aunque correlaciona, no debería ser usado como único criterio.

# Creación del modelo base

In [38]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Other,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,nan,321403,HS-grad,9,Widowed,nan,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


# Evaluación del sesgo en el modelo

In [39]:
X = df.drop("income", axis=1)
y = df["income"]

# Variables categóricas y numéricas
categorical = X.select_dtypes(include=["object"]).columns
numeric = X.select_dtypes(exclude=["object"]).columns

# --- PREPROCESAMIENTO ---
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
    ]
)

# --- PIPELINE ---
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=500))
])

# --- SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- ENTRENAR ---
model.fit(X_train, y_train)

# --- EVALUACIÓN ---
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# --- EXPORTAR ---
joblib.dump(model, "modelo_income.pkl")
print("✅ Modelo entrenado y guardado")

              precision    recall  f1-score   support

       <=50K       0.88      0.94      0.91      7414
        >50K       0.74      0.59      0.66      2355

    accuracy                           0.85      9769
   macro avg       0.81      0.76      0.78      9769
weighted avg       0.84      0.85      0.85      9769

✅ Modelo entrenado y guardado


- **Accuracy (0.85)**
    El modelo tiene un buen desempeño general: 85% de aciertos en el conjunto de prueba. Sin embargo, la exactitud global puede ser engañosa, ya que esconde desigualdades entre grupos que se podrá ver evidenciado en el análisis por sexo y raza.

- **Precision (>50K = 0.74)**

    Cuando el modelo predice que una persona gana >50K, acierta el 74% de las veces. Esto refleja que el modelo es relativamente confiable para asignar la clase positiva, lo cual es importante si se usa para tomar decisiones.

- **Recall (>50K = 0.59)**

    El recall para la clase >50K es bajo ya que solo identifica correctamente al 59% de las personas que realmente ganan más de 50K. Dicho de otra forma, el modelo deja fuera al 41% de los verdaderos positivos (falsos negativos). Esto implica que muchas personas que sí cumplen el criterio de ingresos no son reconocidas.

- **F1-score (>50K = 0.66)**

    El F1 combina precisión y recall, y en este caso es moderado (0.66). Esto indica un balance aceptable entre predecir con certeza y no dejar tantos casos sin reconocer, pero sigue 
    mostrando que el modelo tiene dificultades para capturar a todos los positivos.

- **Desbalance de clases**

    También se observa que la clase <=50K tiene métricas mucho más altas (accuracy 0.94, recall 0.94, F1=0.91) que la clase >50K. Esto es esperado, ya que el dataset Adult está desbalanceado
    (más casos de <=50K), pero refuerza que el modelo tiende a favorecer la clase mayoritaria.

In [40]:
POS_LABEL = ">50K"

def metricas_por_grupo(X, y_true, y_pred, atributo_sensible):
    resultados = {}
    for grupo in X[atributo_sensible].dropna().unique():
        idx = (X[atributo_sensible] == grupo).values
        if idx.sum() == 0:
            continue
        acc = accuracy_score(y_true[idx], y_pred[idx])
        prec = precision_score(y_true[idx], y_pred[idx], pos_label=POS_LABEL, zero_division=0)
        rec = recall_score(y_true[idx], y_pred[idx], pos_label=POS_LABEL, zero_division=0)
        f1 = f1_score(y_true[idx], y_pred[idx], pos_label=POS_LABEL, zero_division=0)
        resultados[grupo] = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}
    return pd.DataFrame(resultados).T.sort_index()

def tasas_y_fairness(X, y_true, y_pred, atributo_sensible):
    df = X[[atributo_sensible]].copy()
    df["y_true"] = y_true.values
    df["y_pred"] = y_pred

    filas = []
    for grupo, dfg in df.groupby(atributo_sensible):
        sr = np.mean(dfg["y_pred"] == POS_LABEL)

        # TPR
        pos_mask = dfg["y_true"] == POS_LABEL
        tpr = np.mean(dfg.loc[pos_mask, "y_pred"] == POS_LABEL) if pos_mask.any() else np.nan

        # FPR
        neg_mask = dfg["y_true"] != POS_LABEL
        fpr = np.mean(dfg.loc[neg_mask, "y_pred"] == POS_LABEL) if neg_mask.any() else np.nan

        filas.append({"grupo": grupo, "selection_rate": sr, "TPR": tpr, "FPR": fpr})

    out = pd.DataFrame(filas).set_index("grupo").sort_index()

    # disparate impact vs grupo con mayor selection_rate (referencia)
    ref_grupo = out["selection_rate"].idxmax()
    ref_val = out.loc[ref_grupo, "selection_rate"]
    out["disparate_impact_vs_max"] = out["selection_rate"] / (ref_val if ref_val > 0 else np.nan)

    # equal opportunity difference (rango de TPR)
    equal_opp_diff = out["TPR"].max() - out["TPR"].min()

    return out, ref_grupo, equal_opp_diff

def graficar_tasas(df_rates, atributo_sensible, carpeta="fairness_plots"):
    Path(carpeta).mkdir(parents=True, exist_ok=True)
    for col in ["selection_rate", "TPR", "FPR"]:
        if col not in df_rates.columns:
            continue
        plt.figure(figsize=(6,4))  # tamaño más pequeño
        df_rates[col].plot(kind="bar", color="pink", edgecolor="black")
        plt.title(f"{col} por {atributo_sensible}")
        plt.xlabel(atributo_sensible)
        plt.ylabel(col)
        plt.tight_layout()
        plt.xticks(rotation=45)
        ruta = f"{carpeta}/{col}_por_{atributo_sensible}.png"
        plt.savefig(ruta, dpi=150)
        plt.show()

model = joblib.load("modelo_income.pkl")
y_pred = model.predict(X_test)

# 2) MÉTRICAS por género
atributo = "sex"
metrics_sex = metricas_por_grupo(X_test, y_test, y_pred, atributo)
print("Métricas por sexo:")
display(metrics_sex)


rates_sex, ref_grp_sex, eq_opp_diff_sex = tasas_y_fairness(X_test, y_test, y_pred, atributo)
print("\nTasas y fairness por sexo:")
display(rates_sex)
print(f"Grupo de referencia (selection_rate más alto): {ref_grp_sex}")
print(f"Equal Opportunity Difference (rango TPR): {eq_opp_diff_sex:.4f}")
graficar_tasas(rates_sex, atributo)

# 3) MÉTRICAS por raza
atributo = "race"
metrics_race = metricas_por_grupo(X_test, y_test, y_pred, atributo)
print("\nMétricas por raza:")
display(metrics_race)

rates_race, ref_grp_race, eq_opp_diff_race = tasas_y_fairness(X_test, y_test, y_pred, atributo)
print("\nTasas y fairness por raza:")
display(rates_race)
print(f"Grupo de referencia (selection_rate más alto): {ref_grp_race}")
print(f"Equal Opportunity Difference (rango TPR): {eq_opp_diff_race:.4f}")
graficar_tasas(rates_race, atributo)


Métricas por sexo:


,accuracy,precision,recall,f1
Female,0.92824,0.785408,0.501370,0.612040
Male,0.81380,0.738433,0.601508,0.662974



Tasas y fairness por sexo:


,selection_rate,TPR,FPR,disparate_impact_vs_max
grupo,,,,
Female,0.072069,0.501370,0.017434,0.290589
Male,0.248011,0.601508,0.093269,1.000000


Grupo de referencia (selection_rate más alto): Male
Equal Opportunity Difference (rango TPR): 0.1001


...

,accuracy,precision,recall,f1
Amer-Indian-Eskimo,0.927083,0.600000,0.375000,0.461538
Asian-Pac-Islander,0.833333,0.703704,0.662791,0.682635
Black,0.904311,0.702703,0.429752,0.533333
Other,0.880597,0.500000,0.250000,0.333333
White,0.845268,0.749112,0.593809,0.662480



Tasas y fairness por raza:


,selection_rate,TPR,FPR,disparate_impact_vs_max
grupo,,,,
Amer-Indian-Eskimo,0.052083,0.375000,0.022727,0.204475
Asian-Pac-Islander,0.254717,0.662791,0.103448,1.000000
Black,0.077813,0.429752,0.026506,0.305487
Other,0.059701,0.250000,0.033898,0.234384
White,0.202711,0.593809,0.068332,0.795828


Grupo de referencia (selection_rate más alto): Asian-Pac-Islander
Equal Opportunity Difference (rango TPR): 0.4128


C:\Users\daher\AppData\Local\Temp\ipykernel_36700\3354708380.py:61: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\daher\AppData\Local\Temp\ipykernel_36700\3354708380.py:61: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\daher\AppData\Local\Temp\ipykernel_36700\3354708380.py:61: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


📊 Análisis por Sexo

- **Selection Rate por sexo**

    La tasa de selección muestra que el modelo clasifica como >50K al 24.8% de los hombres y solo al 7.2% de las mujeres. Esto significa que el modelo considera con mucha mayor frecuencia que los hombres tienen ingresos altos, reforzando un sesgo en contra de las mujeres.

- **TPR (True Positive Rate) por sexo**

    El recall (TPR) de los hombres es 0.60, mientras que el de las mujeres es 0.50. Esto implica que el modelo reconoce mejor a los hombres que realmente ganan >50K que a las mujeres en la misma situación. Aunque la diferencia parece moderada, es significativa, pues refleja una oportunidad desigual de ser correctamente identificado como de ingresos altos.

- **FPR (False Positive Rate) por sexo**

    Los hombres tienen un FPR mucho más alto (0.093) que las mujeres (0.017). Es decir, el modelo tiende a “sobreclasificar” a los hombres como >50K incluso cuando no lo son, mientras que con las mujeres es mucho más restrictivo. Esto refuerza la conclusión de que el modelo favorece a los hombres, otorgándoles más clasificaciones positivas, aun a costa de equivocarse más.

📊 Análisis por Raza

- **Selection Rate por raza**

    El grupo con mayor tasa de selección es Asian-Pac-Islander (25.4%), seguido por White (20.3%). En contraste, grupos como Amer-Indian-Eskimo (5.2%) y Other (5.9%) casi nunca son clasificados como >50K. Esto muestra un claro desequilibrio en las oportunidades de ser reconocidos como de altos ingresos, lo cual es un reflejo de sesgo estructural.

- **TPR (True Positive Rate) por raza**

    El modelo tiene un recall muy alto para Asian-Pac-Islander (0.66) y White (0.59), lo que significa que identifica correctamente a una buena parte de quienes ganan >50K en esos grupos. Sin embargo, para Other (0.25) y Amer-Indian-Eskimo (0.38), el recall cae a niveles críticos, implicando que estos grupos tienen muy baja probabilidad de ser reconocidos correctamente cuando realmente tienen ingresos altos. La diferencia entre máximos y mínimos (Equal Opportunity Difference de 0.41) refleja una desigualdad muy fuerte.

- **FPR (False Positive Rate) por raza**

    Los grupos Asian-Pac-Islander (0.10) y White (0.07) tienen tasas de falsos positivos bastante más altas que las de Amer-Indian-Eskimo (0.02) y Black (0.03). Esto significa que al modelo le resulta más “fácil” dar oportunidades de clasificación positiva a los primeros, incluso cuando no corresponde, mientras que los grupos minoritarios no solo reciben menos oportunidades, sino que además se les clasifica con más “rigor”. Esto revela una asimetría en el trato entre razas.

In [41]:
# --- SHAP: preparar datos y explainer (LogisticRegression) ---
import shap, numpy as np
from scipy import sparse
from sklearn.pipeline import Pipeline

prep = model.named_steps["preprocessor"]
clf  = model.named_steps["classifier"]

def feat_names(prep, orig_cols):
    names=[]
    for _, trf, cols in prep.transformers_:
        if trf == "drop": 
            continue
        if isinstance(trf, Pipeline):
            trf = trf[-1]
        names += (trf.get_feature_names_out(cols).tolist()
                  if hasattr(trf, "get_feature_names_out") else list(cols))
    return names

Xtr = prep.transform(X_train)
Xte = prep.transform(X_test)
feature_names = feat_names(prep, X_train.columns)

def to_dense(X):
    return X.toarray() if sparse.issparse(X) else np.asarray(X)

# background pequeño (usa .shape[0] para sparse)
rng = np.random.default_rng(42)
n = Xtr.shape[0]
idx = rng.choice(n, size=min(200, n), replace=False)

bg_dense   = to_dense(Xtr[idx])
Xte_dense  = to_dense(Xte)

# ✅ API nueva: usa un masker
masker = shap.maskers.Independent(bg_dense)

# Opción 1 (recomendada en 0.48): auto-selección con algoritmo lineal
explainer = shap.Explainer(clf, masker, algorithm="linear")

# Opción 2 (también válida): LinearExplainer sin feature_dependence
# explainer = shap.LinearExplainer(clf, masker)

shap_expl = explainer(Xte_dense)

# índice local interesante (error o cerca del umbral)
proba   = model.predict_proba(X_test)[:, 1]
y_pred  = model.predict(X_test)
y_true  = y_test.values
errs    = np.where(y_pred != y_true)[0]
i_local = int(errs[0]) if len(errs) else int(np.argmin(np.abs(proba - 0.5)))


In [42]:
# ==== SHAP (compacto y actualizado para SHAP>=0.48) ====
import shap, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import sparse
from sklearn.pipeline import Pipeline

# 1) Preparación (usa tu pipeline existente)
prep = model.named_steps["preprocessor"]
clf  = model.named_steps["classifier"]

def feat_names(prep, orig_cols):
    names=[]
    for _, trf, cols in prep.transformers_:
        if trf == "drop": 
            continue
        if isinstance(trf, Pipeline):
            trf = trf[-1]
        if hasattr(trf, "get_feature_names_out"):
            names += trf.get_feature_names_out(cols).tolist()
        else:
            names += list(cols)
    return names

Xtr = prep.transform(X_train)
Xte = prep.transform(X_test)
feature_names = feat_names(prep, X_train.columns)

def to_dense(X):
    return X.toarray() if sparse.issparse(X) else np.asarray(X)

# Background pequeño (usa .shape[0] para sparse)
rng = np.random.default_rng(42)
n = Xtr.shape[0]
row_bg = rng.choice(n, size=min(200, n), replace=False)
bg_dense  = to_dense(Xtr[row_bg])
Xte_dense = to_dense(Xte)

# Explainer lineal con masker (API nueva)
masker = shap.maskers.Independent(bg_dense)
explainer = shap.Explainer(clf, masker, algorithm="linear")
shap_expl = explainer(Xte_dense)

# 🔹 Agregar nombres y valores al Explanation para que los plots muestren labels reales
shap_expl.feature_names = feature_names
shap_expl.data = Xte_dense

# Ejemplo local (error, o cerca del umbral 0.5)
proba   = model.predict_proba(X_test)[:, 1]
y_pred  = model.predict(X_test)
y_true  = y_test.values
errs    = np.where(y_pred != y_true)[0]
i_local = int(errs[0]) if len(errs) else int(np.argmin(np.abs(proba - 0.5)))

# 2) Gráficos globales
shap.plots.beeswarm(shap_expl, show=False)
plt.title("Beeswarm – Global"); plt.tight_layout()
plt.savefig(FIG_DIR / "shap_beeswarm.png", dpi=150); plt.close()

shap.plots.bar(shap_expl, show=False)
plt.title("Bar – |SHAP| medio"); plt.tight_layout()
plt.savefig(FIG_DIR / "shap_bar.png", dpi=150); plt.close()

# 3) Decision plot (submuestreo para evitar RuntimeError)
vals = shap_expl.values
base = shap_expl.base_values.mean() if shap_expl.base_values is not None else 0.0
top_idx = np.argsort(-np.abs(vals).mean(0))[:12]
feat_top = [feature_names[i] for i in top_idx]

impact = np.abs(vals).sum(axis=1)                           # filas más “informativas”
row_idx = np.argsort(-impact)[:min(400, vals.shape[0])]     # <=400 obs
shap.decision_plot(base, vals[row_idx][:, top_idx], feature_names=feat_top, ignore_warnings=True, show=False)
plt.title("Decision plot – Top 12 (subsample)"); plt.tight_layout()
plt.savefig(FIG_DIR / "shap_decision_plot.png", dpi=150); plt.close()

# 4) Waterfall local (usará feature_names y data)
shap.plots.waterfall(shap_expl[i_local], show=False)
plt.title(f"Waterfall – idx={i_local}"); plt.tight_layout()
plt.savefig(FIG_DIR / f"shap_waterfall_{i_local}.png", dpi=150); plt.close()

# 5) Dependence plots (usando índices en vez de strings)
for f in ("age", "hours-per-week", "education-num"):
    if f in feature_names:
        idx = feature_names.index(f)
        shap.plots.scatter(shap_expl[:, idx], show=False)
        plt.title(f"Dependence – {f}")
        plt.tight_layout()
        plt.savefig(FIG_DIR / f"shap_dependence_{f}.png", dpi=150)
        plt.close()

# 6) Fairness con SHAP (por sex / race)
def abs_by_group(expl, feat_names, groups):
    df = pd.DataFrame(np.abs(expl.values), columns=feat_names)
    df["__group__"] = groups.values
    return df.groupby("__group__")[feat_names].mean().T

def margin_by_group(expl, groups):
    margins = expl.values.sum(axis=1)
    return pd.DataFrame({"group": groups.values, "margin": margins}).groupby("group")["margin"].agg(["mean", "median", "count"])

for s in ("sex", "race"):
    if s not in X_test.columns:
        continue
    g = X_test[s]

    tab_abs = abs_by_group(shap_expl, feature_names, g)
    tab_abs.to_csv(DL_DIR / f"shap_abs_{s}.csv")
    tab_abs.head(15).plot(kind="bar", figsize=(10,4))
    plt.title(f"|SHAP| medio por {s} (Top 15)"); plt.tight_layout()
    plt.savefig(FIG_DIR / f"shap_abs_{s}_bar.png", dpi=150); plt.close()

    tab_margin = margin_by_group(shap_expl, g)
    tab_margin.to_csv(DL_DIR / f"shap_margin_{s}.csv")
    tab_margin["mean"].plot(kind="bar")
    plt.title(f"Margen SHAP medio por {s}"); plt.tight_layout()
    plt.savefig(FIG_DIR / f"shap_margin_{s}_bar.png", dpi=150); plt.close()

print("✅ SHAP listo. Figuras en", FIG_DIR, "y CSV en", DL_DIR)

# 7) Gráficas interactivas (HTML para GitHub Pages) con nombres reales
# Casos: error (si hay), cercano a 0.5, acierto confiado
errs = np.where(y_pred != y_true)[0]
i_err = int(errs[0]) if len(errs) else None
i_thresh = int(np.argmin(np.abs(proba - 0.5)))
correct_idx = np.where(y_pred == y_true)[0]
i_conf = int(correct_idx[np.argmax(np.abs(proba[correct_idx] - 0.5))]) if len(correct_idx) else None

# Force plots locales con feature_names y features (Xte_dense)
if i_err is not None:
    shap.save_html(str(DL_DIR / "shap_force_error.html"),
                   shap.plots.force(shap_expl[i_err], feature_names=feature_names, features=Xte_dense[i_err]))
shap.save_html(str(DL_DIR / "shap_force_threshold.html"),
               shap.plots.force(shap_expl[i_thresh], feature_names=feature_names, features=Xte_dense[i_thresh]))
if i_conf is not None:
    shap.save_html(str(DL_DIR / "shap_force_confident.html"),
                   shap.plots.force(shap_expl[i_conf], feature_names=feature_names, features=Xte_dense[i_conf]))

print("✅ Force plots locales (con nombres) guardados en:", DL_DIR)

# Force plot global interactivo (submuestreo de observaciones más “informativas”)
impact = np.abs(vals).sum(axis=1)
take = min(500, vals.shape[0])
row_idx = np.argsort(-impact)[:take]

shap.save_html(
    str(DL_DIR / "shap_force_global.html"),
    shap.plots.force(shap_expl[row_idx], feature_names=feature_names, features=Xte_dense[row_idx])
)
print("✅ Force plot global (con nombres) guardado en:", DL_DIR)


✅ SHAP listo. Figuras en docs\assets\figures y CSV en docs\assets\downloads
✅ Force plots locales (con nombres) guardados en: docs\assets\downloads
✅ Force plot global (con nombres) guardado en: docs\assets\downloads


# Propuesta de mitigación